In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("train.csv")
df

,Unnamed: 0,Text,label
0,0,I can't shake off this constant sense of hopel...,Depression
1,1,I'm constantly second-guessing myself and my d...,Anxiety Disorder
2,2,"I'm feeling physically unwell, but I know it's...",Depression
3,3,I'm desperate to escape the overwhelming fear.,Panic Disorder
4,4,It's hard to describe the sensation of being t...,Panic Disorder
...,...,...,...
780,780,I'm not good enough for anything.,Depression
781,781,"Feeling like I'm a trailblazer, forging a path...",Narcissistic Disorder
782,782,Trying to maintain my self-assuredness without...,Narcissistic Disorder
783,783,Feeling like I have the potential to influence...,Narcissistic Disorder


In [3]:
df.drop(['Unnamed: 0'],axis=1,inplace=True)
df

,Text,label
0,I can't shake off this constant sense of hopel...,Depression
1,I'm constantly second-guessing myself and my d...,Anxiety Disorder
2,"I'm feeling physically unwell, but I know it's...",Depression
3,I'm desperate to escape the overwhelming fear.,Panic Disorder
4,It's hard to describe the sensation of being t...,Panic Disorder
...,...,...
780,I'm not good enough for anything.,Depression
781,"Feeling like I'm a trailblazer, forging a path...",Narcissistic Disorder
782,Trying to maintain my self-assuredness without...,Narcissistic Disorder
783,Feeling like I have the potential to influence...,Narcissistic Disorder


In [4]:
Label = list(df.label.unique())

In [5]:
Label

['Depression',
 'Anxiety Disorder',
 'Panic Disorder',
 'Anger/ Intermittent Explosive Disorder',
 'Narcissistic Disorder']

In [6]:
dic = {'Depression' : 0, 'Anxiety Disorder' : 1, 'Panic Disorder' : 2,
       'Anger/ Intermittent Explosive Disorder' : 3, 'Narcissistic Disorder' : 4}
df.label = df.label.map(dic)
df

,Text,label
0,I can't shake off this constant sense of hopel...,0
1,I'm constantly second-guessing myself and my d...,1
2,"I'm feeling physically unwell, but I know it's...",0
3,I'm desperate to escape the overwhelming fear.,2
4,It's hard to describe the sensation of being t...,2
...,...,...
780,I'm not good enough for anything.,0
781,"Feeling like I'm a trailblazer, forging a path...",4
782,Trying to maintain my self-assuredness without...,4
783,Feeling like I have the potential to influence...,4


In [7]:
import contractions as con
import re
contraction = "[A-Za-z]*'[A-Za-z]{1,2}"

In [8]:
for i in range(len(df)):
    t = df.loc[i,'Text']
    L = re.findall(string=t,pattern=contraction)
    if len(L)>0:
        for word in L:
            df.loc[i,'Text'] = re.sub(re.escape(word),con.fix(word), t)
df

,Text,label
0,I cannot shake off this constant sense of hope...,0
1,I am constantly second-guessing myself and my ...,1
2,"I'm feeling physically unwell, but I know it i...",0
3,I am desperate to escape the overwhelming fear.,2
4,It is hard to describe the sensation of being ...,2
...,...,...
780,I am not good enough for anything.,0
781,"Feeling like I am a trailblazer, forging a pat...",4
782,Trying to maintain my self-assuredness without...,4
783,Feeling like I have the potential to influence...,4


In [9]:
from tensorflow.keras.utils import to_categorical

In [10]:
Y = to_categorical(df['label'])
Y

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [11]:
from transformers import AutoTokenizer,TFBertModel

C:\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert = TFBertModel.from_pretrained('bert-base-cased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [13]:
tokenizer

BertTokenizerFast(name_or_path='bert-base-cased', vocab_size=28996, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [14]:
bert

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
x_train,x_test,y_train,y_test = train_test_split(df['Text'],Y,random_state=42,test_size=0.2)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(628,)
(628, 5)
(157,)
(157, 5)


In [17]:
max_len = 0
for each in df['Text']:
    max_len = max(max_len,len(each))
max_len

86

In [18]:
X_train = tokenizer(
    text = df['Text'].tolist(),
    add_special_tokens=True,
    max_length=max_len,
    truncation=True,
    return_tensors='tf',
    return_token_type_ids=False,
    return_attention_mask=True,
    verbose=True,
    padding=True
)

X_test = tokenizer(
    text = x_test.tolist(),
    add_special_tokens=True,
    max_length=max_len,
    truncation=True,
    return_tensors='tf',
    return_token_type_ids=False,
    return_attention_mask=True,
    verbose=True,
    padding = True
)


In [19]:
X_train

{'input_ids': <tf.Tensor: shape=(785, 26), dtype=int32, numpy=
array([[  101,   146,  2834, ...,     0,     0,     0],
       [  101,   146,  1821, ...,     0,     0,     0],
       [  101,   146,   112, ...,     0,     0,     0],
       ...,
       [  101, 19020,  1106, ...,     0,     0,     0],
       [  101, 13085,  1176, ...,     0,     0,     0],
       [  101,   146,  1821, ...,     0,     0,     0]])>, 'attention_mask': <tf.Tensor: shape=(785, 26), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>}

# Model Building

In [20]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input,Dense,GlobalMaxPool1D,Dropout

In [26]:
max_len=26
input_ids = Input(shape=(max_len,),dtype = tf.int32,name='input_ids')
input_mask = Input(shape=(max_len,),dtype = tf.int32,name="attention_mask")
embeddings = bert(input_ids,attention_mask=input_mask)[0]
out = GlobalMaxPool1D()(embeddings)
out = Dense(128,activation='relu')(out)
out = Dropout(0.9)(out)
out = Dense(32,activation='relu')(out)
output = Dense(5,activation='softmax')(out)

model = tf.keras.Model(inputs=[input_ids,input_mask],outputs=output)
#model.layers[2].trainable=True

In [27]:
print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 26)]                 0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, 26)]                 0         []                            
 )                                                                                                
                                                                                                  
 tf_bert_model (TFBertModel  TFBaseModelOutputWithPooli   1083102   ['input_ids[0][0]',           
 )                           ngAndCrossAttentions(last_   72         'attention_mask[0][0]']      
                             hidden_state=(None, 26, 76                                     

In [28]:
model.compile(optimizer=Adam(learning_rate=5e-5,epsilon=1e-8,clipnorm=1.0),
             loss=CategoricalCrossentropy(from_logits=True),
             metrics=CategoricalAccuracy())

In [29]:
epochs=0

In [30]:
epoch=50
model.fit(x={'input_ids':X_train['input_ids'],'attention_mask':X_train['attention_mask']},
         y=Y,
         epochs=epoch,
          batch_size=36
         )
epochs+=epoch

Epoch 1/50


C:\Python310\lib\site-packages\keras\src\backend.py:5562: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


 3/22 [===>..........................] - ETA: 4:07 - loss: 3.3274 - categorical_accuracy: 0.2315

KeyboardInterrupt: 

In [92]:
epochs

91

In [114]:
test = pd.read_csv("test.csv")
test

,id,Text
0,0,I'm worried about the impact of panic on my re...
1,1,Trying to maintain my equilibrium when emotion...
2,2,Feeling like I'm in a constant struggle to fin...
3,3,I'm quick to dismiss others' opinions as less ...
4,4,I hate the person I become when I'm consumed b...
...,...,...
238,238,The pressure to avoid mistakes can keep me fro...
239,239,"Even when challenges arise, I remain unwaverin..."
240,240,It's like my mind is a radio tuned to the freq...
241,241,It's hard to explain the surge of panic that c...


In [115]:
Text = tokenizer(
    text = test['Text'].tolist(),
    add_special_tokens=True,
    max_length=26,
    truncation=True,
    return_tensors='tf',
    return_token_type_ids=False,
    return_attention_mask=True,
    verbose=True,
    padding='max_length'
)

In [116]:
Text['input_ids']

<tf.Tensor: shape=(243, 26), dtype=int32, numpy=
array([[  101,   146,   112, ...,     0,     0,     0],
       [  101, 19020,  1106, ...,     0,     0,     0],
       [  101, 13085,  1176, ...,     0,     0,     0],
       ...,
       [  101,  1135,   112, ...,     0,     0,     0],
       [  101,  1135,   112, ...,     0,     0,     0],
       [  101,  1135,   112, ...,     0,     0,     0]])>

In [117]:
y_pred=model.predict({'input_ids':Text['input_ids'],'attention_mask':Text['attention_mask']})
y_pred

8/8 [==============================] - 33s 3s/step


array([[2.8101492e-06, 5.0888357e-05, 9.9986196e-01, 6.8223941e-05,
        1.6038011e-05],
       [8.4649218e-06, 2.4703340e-07, 2.9002297e-07, 9.9996936e-01,
        2.1696605e-05],
       [2.6162170e-06, 3.2691940e-05, 7.5026278e-06, 1.5292717e-06,
        9.9995565e-01],
       ...,
       [1.9056872e-05, 9.9873358e-01, 1.1990421e-03, 1.8464565e-05,
        2.9831504e-05],
       [3.8530590e-05, 9.9819499e-01, 1.6784118e-03, 3.6616697e-05,
        5.1412666e-05],
       [3.6207286e-03, 5.4287932e-05, 3.0200978e-05, 9.9528509e-01,
        1.0097342e-03]], dtype=float32)

In [97]:
import numpy as np

In [118]:
Y_pred = np.argmax(y_pred,axis=1)
Y_pred

array([2, 3, 4, 4, 3, 4, 4, 3, 2, 4, 0, 1, 3, 2, 4, 1, 3, 2, 0, 2, 1, 0,
       3, 0, 1, 3, 4, 2, 0, 3, 4, 3, 4, 2, 1, 0, 0, 0, 4, 3, 4, 4, 2, 3,
       2, 1, 0, 2, 4, 0, 3, 4, 0, 0, 0, 1, 4, 4, 3, 2, 4, 0, 2, 1, 1, 0,
       4, 2, 0, 2, 4, 0, 0, 1, 2, 4, 2, 2, 4, 0, 0, 1, 1, 3, 2, 4, 1, 3,
       4, 0, 1, 4, 1, 1, 0, 0, 2, 2, 2, 0, 4, 3, 4, 1, 3, 4, 0, 0, 2, 4,
       2, 0, 1, 1, 4, 3, 4, 0, 3, 1, 1, 3, 1, 1, 3, 3, 0, 1, 1, 1, 1, 3,
       2, 4, 3, 2, 4, 1, 1, 0, 4, 0, 1, 2, 1, 3, 4, 1, 0, 1, 0, 1, 1, 3,
       1, 1, 0, 1, 3, 4, 2, 2, 0, 0, 2, 1, 3, 0, 1, 3, 0, 1, 1, 0, 0, 3,
       3, 1, 0, 3, 0, 0, 2, 0, 4, 1, 3, 0, 4, 4, 4, 0, 1, 2, 4, 2, 2, 3,
       0, 0, 4, 2, 3, 3, 0, 2, 1, 1, 4, 3, 0, 4, 0, 4, 4, 3, 1, 4, 0, 1,
       4, 2, 0, 0, 0, 1, 1, 0, 3, 2, 2, 4, 4, 2, 2, 0, 0, 2, 1, 4, 1, 1,
       3], dtype=int64)

In [119]:
predictions = [Label[i] for i in Y_pred]
predictions

['Panic Disorder',
 'Anger/ Intermittent Explosive Disorder',
 'Narcissistic Disorder',
 'Narcissistic Disorder',
 'Anger/ Intermittent Explosive Disorder',
 'Narcissistic Disorder',
 'Narcissistic Disorder',
 'Anger/ Intermittent Explosive Disorder',
 'Panic Disorder',
 'Narcissistic Disorder',
 'Depression',
 'Anxiety Disorder',
 'Anger/ Intermittent Explosive Disorder',
 'Panic Disorder',
 'Narcissistic Disorder',
 'Anxiety Disorder',
 'Anger/ Intermittent Explosive Disorder',
 'Panic Disorder',
 'Depression',
 'Panic Disorder',
 'Anxiety Disorder',
 'Depression',
 'Anger/ Intermittent Explosive Disorder',
 'Depression',
 'Anxiety Disorder',
 'Anger/ Intermittent Explosive Disorder',
 'Narcissistic Disorder',
 'Panic Disorder',
 'Depression',
 'Anger/ Intermittent Explosive Disorder',
 'Narcissistic Disorder',
 'Anger/ Intermittent Explosive Disorder',
 'Narcissistic Disorder',
 'Panic Disorder',
 'Anxiety Disorder',
 'Depression',
 'Depression',
 'Depression',
 'Narcissistic Disord

In [120]:
final = pd.DataFrame()
final['id'] = test['id']
final['label'] = predictions
final

,id,label
0,0,Panic Disorder
1,1,Anger/ Intermittent Explosive Disorder
2,2,Narcissistic Disorder
3,3,Narcissistic Disorder
4,4,Anger/ Intermittent Explosive Disorder
...,...,...
238,238,Anxiety Disorder
239,239,Narcissistic Disorder
240,240,Anxiety Disorder
241,241,Anxiety Disorder


In [121]:
final.to_csv('final6.csv',index=False)